# Output to CSV

In [1]:
from __future__ import print_function

In [2]:
from salib import extend, import_notebooks
import_notebooks()
from Tables import Table

In [3]:
from Frame2D_Base import Frame2D
import Frame2D_Input

In [4]:
##test:
f = Frame2D('frame-1')
f.input_all()
f.rawdata.nodes.data

,NODEID,X,Y
0,A,0,0
1,B,0,4000
2,C,8000,4000
3,D,8000,0


In [5]:
@extend
class Frame2D:
    
    def write_table(self,table_name,ds_name=None,prefix=None,record=True):
        t = getattr(self.rawdata,table_name,None)
        if t is None:
            methodname = 'list_'+table_name
            method = getattr(self,methodname,None)
            if method and callable(method):
                data = method()
                t = Table(table_name,data=data,columns=getattr(self,'COLUMNS_'+table_name))
        if t is None:
            raise ValueError("Unable to find table '{}'".format(table_name))
        t.write(ds_name=ds_name,prefix=prefix)
        if record:
            setattr(self.rawdata,table_name,t)
        return t            
    
    def list_nodes(self):
        return [(n.id,n.x,n.y) for n in self.nodes.values()]

In [6]:
try:
    f.write_table('foobar')
except Exception, e:
    print('**** Error:',e)

**** Error: Unable to find table 'foobar'


In [7]:
##test:
t = f.write_table('nodes','test-1')
t.file_name

'test-1.d/nodes.csv'

In [8]:
##test:
%cat test-1.d/nodes.csv

NODEID,X,Y
A,0.0,0.0
B,0.0,4000.0
C,8000.0,4000.0
D,8000.0,0.0


In [9]:
##test:
if hasattr(f.rawdata,'nodes'):
    delattr(f.rawdata,'nodes')
%rm test-1.d/nodes.csv
t = f.write_table('nodes','test-1')
t.file_name

'test-1.d/nodes.csv'

In [10]:
##test:
%cat test-1.d/nodes.csv

NODEID,X,Y
A,0.0,0.0
B,0.0,4000.0
C,8000.0,4000.0
D,8000.0,0.0


In [11]:
@extend
class Frame2D:
    
    def list_supports(self):
        ans = []
        for node in self.nodes.values():
            if node.constraints:
                cl = tuple(node.constraints)
                if len(cl) < 3:
                    cl = cl + ('',)*(3-len(cl))
                ans.append((node.id,)+cl)
        return ans

In [12]:
##test:
f.list_supports()

[('A', 'FX', 'FY', 'MZ'), ('D', 'FX', 'FY', '')]

In [13]:
##test:
t = f.write_table('supports','test-1')
t.file_name

'test-1.d/supports.csv'

In [14]:
##test:
%cat test-1.d/supports.csv

NODEID,C0,C1,C2
A,FX,FY,MZ
D,FX,FY,


In [15]:
##test:
x = 'supports'
if hasattr(f.rawdata,x):
    delattr(f.rawdata,x)
%rm test-1.d/supports.csv
t = f.write_table(x,'test-1')
t.file_name

'test-1.d/supports.csv'

In [16]:
##test:
%cat test-1.d/supports.csv

NODEID,C0,C1,C2
A,FX,FY,MZ
D,FX,FY,


In [17]:
@extend
class Frame2D:
    
    def list_members(self):
        return [(m.id,m.nodej.id,m.nodek.id) for m in self.members.values()]

In [18]:
##test:
f.list_members()

[('AB', 'A', 'B'), ('BC', 'B', 'C'), ('CD', 'C', 'D')]

In [19]:
##test:
f.write_table('members','test-1')
%cat test-1.d/members.csv

MEMBERID,NODEJ,NODEK
AB,A,B
BC,B,C
CD,C,D


In [20]:
@extend
class Frame2D:
    
    def list_releases(self):
        return [(m.id,)+tuple(m.releases) for m in self.members.values() if m.releases]

In [21]:
##test:
f.list_releases()

[('AB', 'MZK'), ('CD', 'MZJ')]

In [22]:
##test:
f.write_table('releases','test-1')
%cat test-1.d/releases.csv

MEMBERID,RELEASE
AB,MZK
CD,MZJ


In [23]:
@extend
class Frame2D:
    
    def list_properties(self):
        return [(m.id,m.size,m.Ix,m.A) for m in self.members.values()]

In [24]:
##test:
f.list_properties()

[('AB', 'W310x97', 222000000.0, 12300.0),
 ('BC', 'W460x106', 488000000.0, 13500.0),
 ('CD', '', 222000000.0, 12300.0)]

In [25]:
##test:
f.write_table('properties','test-1')
%cat test-1.d/properties.csv

MEMBERID,SIZE,IX,A
BC,W460x106,488000000.0,13500.0
AB,W310x97,222000000.0,12300.0
CD,,222000000.0,12300.0


In [26]:
##test:
vars(f.rawdata).keys()

['member_loads',
 'releases',
 'supports',
 'support_displacements',
 'load_combinations',
 'members',
 'nodes',
 'node_loads',
 'properties']

In [27]:
@extend
class Frame2D:
    
    COLUMNS_signatures = ['BASE','PATH','SIGNATURE']
    
    def list_signatures(self):
        return [t.signature() for tn,t in vars(self.rawdata).items() if type(t) is Table]

In [28]:
##test:
f.list_signatures()

[('member_loads.csv',
  'frame-1.d/member_loads.csv',
  'aa1ce177206060209e11e016c7f4f210e3d3b42bfb6a5965069d5345ffc4628a'),
 ('releases.csv',
  'test-1.d/releases.csv',
  'ae3d4681f60b6c5665cfd07a4894a01dd386cc7a51e98eb08248fc202e41e09f'),
 ('supports.csv',
  'test-1.d/supports.csv',
  'c9b14f3fb0868a679912d6d5d8f23e75f89cc8accd8e9f2b1613bc9710653815'),
 ('support_displacements.csv',
  'frame-1.d/support_displacements.csv',
  'b475cca77a5ddcf815425514c484fcb3506195db840a9bf3006a187c9e457617'),
 ('load_combinations.csv',
  'frame-1.d/load_combinations.csv',
  'efe64703cadacf5f169f913b9d49dcf4c6523dd62e474cfff90782787c830310'),
 ('members.csv',
  'test-1.d/members.csv',
  '783cee2e924a1f1fc738638af431671f02e25f1b97430d834a226eecd3dd09e3'),
 ('nodes.csv',
  'test-1.d/nodes.csv',
  'a1c6202ec4a3f745cbde4e96c7b07ce173d14ebaeaa6956b40260053ffb34e6c'),
 ('node_loads.csv',
  'frame-1.d/node_loads.csv',
  '0e6b29f68205018faf8132182cd8b9ad61360ff3a41c8d67fc9e85c26289ebce'),
 ('properties.csv',


In [30]:
##test:
f.write_table('signatures','test-1',record=False)
%cat test-1.d/signatures.csv

BASE,PATH,SIGNATURE
member_loads.csv,frame-1.d/member_loads.csv,aa1ce177206060209e11e016c7f4f210e3d3b42bfb6a5965069d5345ffc4628a
releases.csv,test-1.d/releases.csv,ae3d4681f60b6c5665cfd07a4894a01dd386cc7a51e98eb08248fc202e41e09f
supports.csv,test-1.d/supports.csv,c9b14f3fb0868a679912d6d5d8f23e75f89cc8accd8e9f2b1613bc9710653815
support_displacements.csv,frame-1.d/support_displacements.csv,b475cca77a5ddcf815425514c484fcb3506195db840a9bf3006a187c9e457617
load_combinations.csv,frame-1.d/load_combinations.csv,efe64703cadacf5f169f913b9d49dcf4c6523dd62e474cfff90782787c830310
members.csv,test-1.d/members.csv,783cee2e924a1f1fc738638af431671f02e25f1b97430d834a226eecd3dd09e3
nodes.csv,test-1.d/nodes.csv,a1c6202ec4a3f745cbde4e96c7b07ce173d14ebaeaa6956b40260053ffb34e6c
node_loads.csv,frame-1.d/node_loads.csv,0e6b29f68205018faf8132182cd8b9ad61360ff3a41c8d67fc9e85c26289ebce
properties.csv,test-1.d/properties.csv,4ebc2477cadb207459034659d0bf1da9b9dbb02bb06af2c060894ecb595effca


In [33]:
@extend
class Frame2D:
    
    def list_node_loads(self):
        ans = []
        dirns = ['FX','FY','FZ']
        for loadid,node,nload in self.nodeloads:
            for i in [0,1,2]:
                if nload[i]:
                    ans.append((loadid,node.id,dirns[i],nload[i]))
        return ans

In [34]:
##test:
f.list_node_loads()

[('wind', 'B', 'FX', -200000.0)]

In [35]:
##test:
f.write_table('node_loads','test-1')
%cat test-1.d/node_loads.csv

LOAD,NODEID,DIRN,F
Wind,B,FX,-200000.0
